In [ ]:
# Authenticate and Initialize Earth Engine
import ee
# Authenticate as your Qwiklabs User
ee.Authenticate()
# Initialize the library by specifying the GCP project
your_project_id = ''
ee.Initialize(project=your_project_id)

In [ ]:
# Import geemap
import geemap.core as geemap

## Basemap selector

Load the geemap viewer. Select a basemap from the dropdown list and add it to the map.

In [ ]:
m = geemap.Map()
m.add("basemap_selector")
m

## Layer Manager

Load an EE Image and a FeatureCollection as a layer. Toggle the checkbox to show or hide the layer. Drag and move the slider to change the transparency level of the layer.

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add_layer(states, {}, "US States")
m.add('layer_manager')
m

## Inspector

Click on the map to query Earth Engine data at a specific location.

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add_layer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
    'Landsat 7',
)
m.add_layer(states, {}, "US States")
m.add('inspector')
m

## Layer Editor

You can change the visualization parameters of the Earth Engine data using the layer editor.

### Single-band image

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add('layer_editor', layer_dict=m.ee_layers['SRTM DEM'])
m

### Multi-band image

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
m.add_layer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
    'Landsat 7',
)
m.add('layer_editor', layer_dict=m.ee_layers['Landsat 7'])
m

### Feature collection

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
m.add_layer(states, {}, "US States")
m.add('layer_editor', layer_dict=m.ee_layers['US States'])
m

## Draw control

You can draw shapes on the map using the draw control. The drawn features will be **automatically** converted to Earth Engine objects, which can be accessed in one of the following ways:

- To return the last drawn feature as an `ee.Geometry()`, use: `m._draw_control.last_geometry`
- To return all the drawn feature as an `ee.Feature()`, use: `m._draw_control.last_feature`
- To return all the drawn features as an `ee.FeatureCollection()`, use: `m._draw_control.collection`

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': 'terrain',
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add('layer_manager')
m

Use the draw control to draw a polygon on the map.

### Explore the EE Geometry, Feature ,and FeatureCollection from your drawn feature


In [ ]:
geometry = m._draw_control.last_geometry
geometry

In [ ]:
feature = m._draw_control.last_feature
feature

In [ ]:
collection = m._draw_control.collection
collection

In [ ]:
if geometry is not None:
    image = dem.clipToCollection(collection)
    m.layers[1].visible = False
    m.add_layer(image, vis_params, "Clipped DEM")
m

### Zonal statistics

Find out which state has the highest mean temperature on in the United States on January 30, 2024. Relevant Earth Engine assets:

- [ee.FeatureCollection("TIGER/2018/States")](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States)
- [ee.ImageCollection("NOAA/GFS0P25")](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25)

![](https://i.imgur.com/GZCHHz3.png)

In [ ]:
m = geemap.Map()

# Load US States
states = ee.FeatureCollection("TIGER/2018/States")

# GFS collects four observations per day. Let's get the first ones after noon
# We'll also get the zero-hour forecast.
startTime = ee.Date('2024-01-29T12:00:00', 'US/Eastern')
endTime = ee.Date(startTime.advance(6, 'hour'))
weather = ee.ImageCollection("NOAA/GFS0P25").filter(ee.Filter.date(startTime, endTime)) \
                                            .filter(ee.Filter.eq('forecast_hours', 0))

temperatureAboveGround = weather.select('temperature_2m_above_ground').first()
visParams = {
    'min': -40.0,
    'max': 35.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

m.addLayer(temperatureAboveGround.clip(states), visParams, 'Temperature Above Ground')
m.addLayer(states, {}, "State Outlines")


states = temperatureAboveGround.reduceRegions(collection=states, reducer=ee.Reducer.mean(), scale=25000)
# Results are returned as a new property called 'mean' on each state.

# This approach is documented here - https://developers.google.com/earth-engine/guides/feature_collections_visualizing
empty = ee.Image().float();
fills = empty.paint(states, 'mean');
m.addLayer(fills, visParams, "Mean Temp")

m.addLayer(states, {}, "State Outlines")

m

In [ ]:
hottest_state = states.sort("mean",False).first()
hottest_state